In [13]:
import sys
import torch
import os
from os import makedirs
import numpy as np
import open3d as o3d
from random import randint

import matplotlib.pyplot as plt

In [14]:
rgb_camera_params = {}
depth_camera_params = {}
relative_positions = {}

def readRGBDConfig(config_file):
    # Define dictionaries to hold camera parameters
    # rgb_camera_params = {}
    # depth_camera_params = {}
    # relative_positions = {}
    
    with open(config_file, 'r') as file:
        data = file.read().split('\n\n')
    
        # Read RGB camera parameters
        rgb_data = data[0].split('\n')
        for line in rgb_data[1:4]:
            key, value = line.split('=')
            if ',' in value:
                value = tuple(map(float, value.split(',')))
            else:
                value = tuple(map(int, value.split('x')))
            rgb_camera_params[key] = value

        vFOV, hFOV = rgb_data[4].split(',')
        key, value = vFOV.split('=')
        rgb_camera_params[key] = float(value.strip('°'))
        key, value = hFOV.split('=')
        rgb_camera_params[key.strip(' ')] = float(value.strip('°'))

        # Read Depth camera parameters
        depth_data = data[1].split('\n')
        for line in depth_data[1:4]:
            key, value = line.split('=')
            if ',' in value:
                value = tuple(map(float, value.split(',')))
            else:
                value = tuple(map(int, value.split('x')))
            depth_camera_params[key] = value

        vFOV, hFOV = depth_data[4].split(',')
        key, value = vFOV.split('=')
        depth_camera_params[key] = float(value.strip('°'))
        key, value = hFOV.split('=')
        depth_camera_params[key.strip(' ')] = float(value.strip('°'))

    
        # Read relative positions of camera components
        rel_pos_data = data[2].split('\n')
        for line in rel_pos_data[1:]:
            key, value = line.split(': ')
            value = tuple(map(float, value.strip('(').strip(')').split(',')))
            relative_positions[key] = value

        # return rgb_camera_params, depth_camera_params, relative_positions
    
    # Access the loaded camera parameters
    print("RGB Camera Parameters:")
    print(rgb_camera_params)
    
    print("\nDepth Camera Parameters:")
    print(depth_camera_params)
    
    print("\nRelative Positions of Camera Components:")
    print(relative_positions)

conifg = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\config\\configuration.txt"
readRGBDConfig(conifg)

RGB Camera Parameters:
{'resolution': (1920, 1080), 'cx,cy,fx,fy': (959.5, 539.5, 960.0, 960.0), 'k1,k2,k3,p1,p2': (0.0, 0.0, 0.0, 0.0, 0.0), 'vFov': 58.7155, 'hFov': 90.0}

Depth Camera Parameters:
{'resolution': (512, 424), 'cx,cy,fx,fy': (255.5, 211.5, 256.0, 256.0), 'k1,k2,k3,p1,p2': (0.0, 0.0, 0.0, 0.0, 0.0), 'vFov': 79.2579, 'hFov': 89.9999}

Relative Positions of Camera Components:
{'RGB sensor': (0.0, 0.0, 0.0), 'RGB light source': (0.0, 0.3, 0.0), 'RGB light source size': (0.4, 0.4), 'Depth sensor': (0.03, 0.0, 0.0), 'Depth light source': (0.06, 0.0, 0.0), 'Depth light source size': (0.02, 0.02)}


In [15]:
# Camera Loader
directory = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\config"

config_files = os.listdir(directory)
config_files.sort()

pose = np.identity(4)

for file in config_files[:1]:
    if file.startswith("campose-rgb-"):
        config_file = os.path.join(directory, file)
        print(config_file)
        
        with open(config_file, 'r') as file:
            lines = file.readlines()

            # Extracting position
            position_str = lines[0].replace('position=', '').split('\n')[0]
            position = np.array([float(i) for i in position_str.strip('()').split(',')])

            # Extracting rotation as a quaternion
            rotation_str = lines[1].replace('rotation_as_quaternion=', '').split('\n')[0]
            rotation = np.array([float(i) for i in rotation_str.strip('()').split(',')])

            # Extracting the 4x4 pose matrix
            pose_str = lines[3:]
            pose = np.array([[float(i) for i in row.strip('(').split(')')[0].split(',')] for row in pose_str if row != ''])    

        print('Position:', position)
        print('Rotation:',rotation)
        print('Pose:',pose)

G:\Universitat Siegen\SA\P-GPU\Code\gaussian-splatting\data\RGBD_Data\config\campose-rgb-0000.txt
Position: [0.5 1.  2.3]
Rotation: [ 1.13133e-08  9.65926e-01  2.58819e-01 -4.22220e-08]
Pose: [[-1.00000e+00  4.37114e-08 -7.57103e-08  5.00000e-01]
 [-6.50875e-16  8.66025e-01  5.00000e-01  1.00000e+00]
 [ 8.74228e-08  5.00000e-01 -8.66025e-01  2.30000e+00]
 [ 0.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00]]


In [16]:
# Directory where your images are stored
directory = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\rgb"

# Get the list of files in the directory
files = os.listdir(directory)

# Sort the files to process depth and color images together
files.sort()

# Initialize an empty point cloud
point_cloud = o3d.geometry.PointCloud()

# Loop through each pair of depth and color images
for file in files[:1]:
    if file.startswith("gt-rgb-depth-"):  # Check if the file is a depth image
        depth_file = os.path.join(directory, file)
        
        # Get the corresponding color image
        color_file = os.path.join(directory, "rgb-" + file[-8:])  # Assuming both files have corresponding indices
        
        # Read the depth and color images
        depth_image = o3d.io.read_image(depth_file)
        color_image = o3d.io.read_image(color_file)
        
        # Convert images to numpy arrays
        depth_array = np.asarray(depth_image)
        color_array = np.asarray(color_image)

        width = depth_array.shape[1]
        height = depth_array.shape[0]
        
        # Intrinsic parameters of the camera (you may need to adjust these values)
        intrinsic = o3d.camera.PinholeCameraIntrinsic()
        cx = rgb_camera_params['cx,cy,fx,fy'][0]
        cy = rgb_camera_params['cx,cy,fx,fy'][1]
        fx = rgb_camera_params['cx,cy,fx,fy'][2] 
        fy = rgb_camera_params['cx,cy,fx,fy'][3]
        intrinsic.set_intrinsics(width=width, height=height, cx=cx, cy=cy, fx=fx, fy=fy)
        
        # Create a point cloud from the depth and color information
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image, depth_trunc=4.0, convert_rgb_to_intensity=False)
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic,extrinsic=pose)
        
        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        
        # Merge current point cloud with the overall point cloud
        point_cloud += pcd

# Visualize the final point cloud
o3d.visualization.draw_geometries([point_cloud])